In [1]:
import sys, os
import pandas as pd
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as influx_Client
from Clust.clust.ML.common.common import p1_integratedDataSaving as p1
db_client = influx_Client.InfluxClient(ins.CLUSTDataServer2)
from Clust.clust.ingestion.mongo import mongo_client
mongo_client = mongo_client.MongoClient(ins.CLUSTMetaInfo2)

In [2]:
# 테스트를 위한 변수 셋팅 및 설정 - 4개의 데이터를 만들어보기 위한 가상 설정
DataMetaPath = "./integratedData.json"
modeList =['trainX', 'trainy', 'testX', 'testy']

clean_param = "NoClean"
bucket_name ="IntDataForRegression"

###
trainStartTime = "2016-01-11"
trainEndTime = "2016-04-15"

testStartTime = "2021-01-01"
testEndTime = "2021-02-12"

trainXDataInfo = [['life_indoor_environment', 'humidityTrain_10min'], 
                  ['life_indoor_environment', 'temperatureTrain_10min'], 
                  ['weather_outdoor_environment', 'belgiumChieverseAirportTrain_10min']]

testXDataInfo = [['life_indoor_environment', 'humidityTest_10min'], 
                 ['life_indoor_environment', 'temperatureTest_10min'], 
                 ['weather_outdoor_environment', 'belgiumChieverseAirportTest_10min']]

In [15]:
for mode in modeList:
    # 0. Set Basic Parameter 
    # 1-1 StartTime, EndTime, DataInfo, Integration_freq_sec, cleanParam, processParam

    if mode == 'trainX':
        dataName = 'IntegraionTrainX'
        start_time = trainStartTime
        end_time = trainEndTime
        dataInfo = trainXDataInfo
        integration_freq_sec = 60 * 10 # 10분

    elif mode =='testX':
        dataName = 'IntegraionTestX'
        start_time = testStartTime
        end_time = testEndTime
        dataInfo = testXDataInfo
        integration_freq_sec = 60 * 10 # 10분

    elif mode == 'trainy':
        dataName = 'IntegraionTrainy'
        start_time = trainStartTime
        end_time = trainEndTime
        dataInfo = [['life_indoor_environment', 'applianceEnergyDatasetTrainy_1day']]
        integration_freq_sec = 60 * 60 * 24 # 24시간
        
    elif mode =='testy':
        dataName = 'IntegraionTesty'
        start_time = testStartTime
        end_time = testEndTime
        dataInfo = [['life_indoor_environment', 'applianceEnergyDatasetTesty_1day']]
        integration_freq_sec = 60 * 60 * 24 # 24시간
    
    # dataset ingestion---> data preprocessing ---> data integration ---> information save
    
    # 1. Ingestion multiple dataset
    #############
    ingestion_param ={}
    ingestion_param['ms_list_info'] = dataInfo
    ingestion_param['start_time'] = start_time
    ingestion_param['end_time'] = end_time
    #############
    
    from Clust.clust.data import data_interface
    multiple_dataset = data_interface.get_data_result("multiple_ms_by_time", db_client, ingestion_param)
    
    # 2. Data Preprocessing
    from Clust.clust.preprocessing import processing_interface
    process_param = p1.get_process_param(clean_param) 
    multiple_dataset = processing_interface.get_data_result('step_3', multiple_dataset, process_param)
    
    # 3. Data Integration
    from Clust.clust.integration.integrationInterface import IntegrationInterface
    integration_param = {
        "integration_frequency":integration_freq_sec,
        "param":{},
        "method":"meta",
        "integration_duration":"common"
    }
    data = IntegrationInterface().multipleDatasetsIntegration(integration_param, multiple_dataset)
    
    # 4. information save
    # mongo db에 process_param, dataInfo, integration_freq_sec, startTime, endTime 정보 적혀야 합니다.
    # data_name이 중복적으로 발생한다는 단점이 있는데, 이는 추후 덮어쓰지 않기 위해서는, 기존에 이런 데이터가 있냐 덮어쓰겠냐?
    # UI적으로 요런 루트만 만들어주면 되지 않을까 싶어요.
    # 위 p1.get_process_param 이부분 나중에 좀 생각해보고 없애겠습니다.
    
    ms_name = "energy" + mode + '_' + "regression" + '_' + clean_param
    db_client.write_db(bucket_name, ms_name, data)
    # save to mongodb
    data_info = p1.integrated_data_meta(dataInfo, start_time, end_time, integration_freq_sec, clean_param, process_param)
    mongo_client.insert_document(bucket_name, collection_name, data_info)

===integrationStart===
===integrationEnd===
========== write success ==========


NameError: name 'collection_name' is not defined